In [4]:
from my_secret_key import mistral_key
import os
os.environ["MISTRAL_API_KEY"] = mistral_key
# print(os.getenv("MISTRAL_API_KEY"))

In [8]:
from langchain_mistralai import ChatMistralAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate
#from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence
#from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain, LLMChain, SequentialChain

In [ ]:
model = ChatMistralAI(model="mistral-large-latest")
parser = StrOutputParser()
# system_template = "Translate the following into {language}:"
# prompt_template = ChatPromptTemplate.from_messages(
#     [("system", system_template), ("user", "{text}")]
# )
# result= prompt_template.invoke({"language":"italian","text":"hi"})
# chain = prompt_template | model | parser
# respose=chain.invoke({"language": "italian", "text": "hi"})
# print(respose )

p1=PromptTemplate(input_variables=['cuisine'],template="I want to open restaturant for {cuisine} food.Suggest only one fancy name for this. Respond only name")
p2=PromptTemplate(input_variables=['restaurant_name'],template="Suggest some menu items for {restaurant_name}. Return it as a comma seprated list.")

# Create chains for generating restaurant names and menu items
name_chain = LLMChain(prompt=p1, llm=model, output_parser=parser)
menu_chain = LLMChain(prompt=p2, llm=model, output_parser=parser)

# result = menu_chain.run(restaturant_name=restaurant_name)
# print(result)
#name_chain = LLMChain(prompt=name_prompt_template, llm=model, output_parser=parser)
#food_chain = LLMChain(prompt=food_prompt_template, llm=model, output_parser=parser)

'''
name_prompt_template = ChatPromptTemplate.from_messages(
    [("user", "I want to open restaturant for {cuisine} food.Suggest only one fancy name for this. Respond only name")]
)
food_prompt_template = ChatPromptTemplate.from_messages(
    [("user", "Suggest some menu items for {restaturant_name}.Return only menu item in comma seprated list.")]
)
chain = name_prompt_template | model | parser
print(chain)

combined_sequence = RunnableSequence(
    {
        "restaturant_name":chain
    },
    food_prompt_template,
    model,
    parser
)

print(combined_sequence)

respose=combined_sequence.invoke({"cuisine": "italian"})
print(f"Restaurant Name: {respose}")'''

In [2]:

# Initialize Mistral or OpenAI LLM (depending on your setup)
# llm = ChatMistralAI(temperature=0.6)  # Replace with Mistral if using Mistral integration

# # Create a chat prompt template
# prompt_template = ChatPromptTemplate.from_template(
#     "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this. Return it as comma seprarted list."
# )

# # Format it with specific inputs
# formatted_prompt = prompt_template.format_messages(cuisine="Indian")

# # Extract the string content from the ChatPromptValue
# prompt_text = formatted_prompt[0].content

# # Pass the string content to the LLM
# response = llm.invoke(prompt_text)
# print(response.content)


In [3]:
chain = SimpleSequentialChain(
    chains=[name_chain, menu_chain],
    verbose=True
)

#menu_items = chain.run('italian')
#print("==========",menu_items)

In [ ]:
p1=PromptTemplate(input_variables=['cuisine'],template="I want to open restaturant for {cuisine} food.Suggest only one fancy name for this. Respond only name")

p2=PromptTemplate(input_variables=['restaurant_name'],template="Suggest some menu items for {restaurant_name}. Return only menu item in comma seprated list")

# Create chains for generating restaurant names and menu items
name_chain = LLMChain(prompt=p1, llm=model, output_parser=parser,output_key='restaurant_name')
menu_chain = LLMChain(prompt=p2, llm=model, output_parser=parser,output_key='menu_items')

chain = SequentialChain(
    chains=[name_chain, menu_chain],
    input_variables=['cuisine'],
    output_variables=['restaurant_name','menu_items'],
    verbose=True
)

menu_items = chain({'cuisine':'italian'})
print("==========",menu_items)

In [ ]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="mistral-large-latest")
tools=load_tools(["wikipedia","llm-math"],llm=model)

agent=initialize_agent(tools,
                 model,
                 agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                 verbose=True
                 )

agent.run("When was elon musk born? what is his age right now?")

In [ ]:
p1=PromptTemplate(input_variables=['cuisine'],template="I want to open restaturant for {cuisine} food.Suggest only one fancy name for this. Respond only name")
parser = StrOutputParser()
name_chain = LLMChain(prompt=p1, llm=model, output_parser=parser,output_key='restaurant_name')
dir(name_chain)